＃コードジェネレーター

要件：フロンティアモデルを使用して、Pythonコードから高性能C ++コードを生成する


<テーブルスタイル= "マージン：0;テキストアライグ：左;">
    <tr>
        <td style = "width：150px; height：150px; vertical-align：middle;">
            <img src = "../ resources.jpg" width = "150" height = "150" style = "display：block;" />
        </td>
        <td>
            <h2 style = "color：＃f71;">リマインダー：最新コードを取得</h2>
            <Span style = "color：＃f71;">これらのラボを継続的に改善し、より多くの例とエクササイズを追加しています。
            毎週の初めに、最新のコードがあることを確認する価値があります。<br/>
            最初にa <a href = "https://chatgpt.com/share/6734E705-3270-8012-A074-421661AF6BA9">必要に応じて変更をプルしてマージします</a>。何か問題がありますか？ chatgptにマージの方法を明確にするように頼んでみてください - または私に連絡してください！<br/> <br/>
            LLM_Engineeringディレクトリからコードをプルした後、Anacondaプロンプト（PC）またはターミナル（MAC）で、実行してください：<br/>
            <code> conda env update  -  f Environment.yml -prune </code> <br/>
            または、AnacondaではなくVirtualenvを使用した場合は、PowerShell（PC）またはターミナル（MAC）でアクティブ化された環境からこれを実行します。<br/>
            <code> pip install -r requistence.txt </code>
            <br/>次に、カーネル（カーネルメニュー>>カーネルの再起動とすべてのセルのクリア出力）を再起動して、変更を拾います。
            </span>
        </td>
    </tr>
</table>

<テーブルスタイル= "マージン：0;テキストアライグ：左;">
    <tr>
        <td style = "width：150px; height：150px; vertical-align：middle;">
            <img src = "../ fality.jpg" width = "150" height = "150" style = "display：block;" />
        </td>
        <td>
            <h1 style = "color：＃900;">重要な注意</h1>
            <Span style = "color：＃900;">
            このラボでは、GPT-4OとClaude-3.5-Sonnetを使用します。これは、わずかに高い価格モデルです。コストはまだ低いですが、コストを非常に低く抑えたい場合は、モデルに推奨されるスイッチを作成してください（ここから3セルダウン）。
            </span>
        </td>
    </tr>
</table>

In [ ]:
# 輸入

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [ ]:
# 環境

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

In [ ]:
# 初期化
# 注 - 最後の2行を除外して超低コストモデルを使用するオプション

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

# コストを非常に低く保ちたいですか？これらの行の解消：
# openai_model = "gpt-4o-mini"
# claude_model = "claude-3-haiku-20240307"

In [ ]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [ ]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to # IOMANIPなどの必要なすべてのC ++パッケージを含めます。\ n \ n "
    user_prompt += python
    return user_prompt

In [ ]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [ ]:
# Optimized.cppというファイルに書き込みます

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [ ]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [ ]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [ ]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
exec(pi)

In [ ]:
optimize_gpt(pi)

In [ ]:
exec(pi)

# C ++をコンパイルして実行します

この次のセルには、M1 MacにC ++ファイルをコンパイルするコマンドが含まれています。  
「最適化された.cpp`」を「最適化」と呼ばれる実行可能ファイルにコンパイルします  
次に、「最適化」と呼ばれるプログラムを実行します

次のラボ（4日目）では、学生がMac、PC、Linuxの効率的なコードにコンパイルする完全なソリューションを提供しました！

これを待つことができます。または、プラットフォームでこれを行う方法についてGoogle Google Google（またはChatGpt！）を使用してから、以下の行を交換できます。
このステップに慣れていない場合は、確かにスキップできます。Macでどのように機能するかを正確に示します。


代わりに：学生Sandeep K.G. PythonとC ++コードをオンラインで実行して、そのようにテストできることを指摘しています。ありがとうSandeep！  
>正確な比較ではありませんが、パフォーマンスの違いのアイデアを得ることができます。
>例：https：//www.programiz.com/cppprogramming/online-compiler/

In [ ]:
# C ++をコンパイルし、実行可能ファイルを実行します

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

In [ ]:
optimize_claude(pi)

In [ ]:
# クロードのために繰り返し - 繰り返しますが、プラットフォームに適切なアプローチを使用してください

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

In [ ]:
python_hard = """# 多数のサイズをサポートするように注意してください

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# パラメーター
n = 10000         # 乱数数
initial_seed = 42 # LCGの初期シード
min_val = -10     # 乱数の最小値
max_val = 10      # 乱数の最大値

# 関数のタイミング
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("総最大サブアレイ合計（20回の実行）：", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
exec(python_hard)

In [ ]:
optimize_gpt(python_hard)

In [ ]:
# これを適切なC ++コンパイル+プラットフォームの実行コマンドに置き換えます

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

In [ ]:
optimize_claude(python_hard)

In [ ]:
# これを適切なC ++コンパイル+プラットフォームの実行コマンドに置き換えます

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

In [ ]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [ ]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [ ]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far        

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

In [ ]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [ ]:
# プラットフォームのC ++コードをコンパイルするために、トライブロックのコードを変更する必要があります
# Intel PCのバージョンを提供するためにリクエストを使用して、ClaudeのチャットUIに貼り付けました。
# そして、それは完璧に見えるもので応答しました - あなたはあなたのプラットフォームに同様のアプローチを試すことができます。

# 最適化されたC ++コードをコンパイルおよび実行するM1 Macバージョン：

def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [ ]:
css = """
.python {background-color: # 306998;}
.cpp {background-color: # 050;}
"""

In [ ]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("# ＃コードをPythonからC ++に変換する "）
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)